In [21]:
import keras
from keras import layers
import numpy as np
import random
import io


In [22]:
path = "text.txt"
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # Eliminar las nuevas lineas
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cortar el texto en bloques iguales
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 2188
Total chars: 43
Number of sequences: 716


In [23]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [24]:
def sample(preds, temperature=1.0):
    # Funcion que permite muestrear 
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [ ]:
epochs = 100
batch_size = 32

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("------- GENERADO DESPUÉS DE LA ÉPOCA: %d -------------------------------------------------------------------------------------------------------------------------------------------------" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("\tDIVERSIDAD: ", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('\t   Generando con semilla: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("\t   Generado --> ", generated)
        print("\t--------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.3924

------- GENERADO DESPUÉS DE LA ÉPOCA: 0 -----------------------------------------------------------------------------------------------------------------------------------------------
	DIVERSIDAD:  0.2
	   Generando con semilla: "iental, ingeniería electrónica, ingenier"
	   Generado -->   n aa r n  c   a     e         n n   r      n   nnna ne n    ta  n   na rnn          l  a r    er nr e a t      re  n n r a     an en     a  n         ra r nn  n    a  a  n   e  r  an a  a  n    r   n    e                n   n a      n ea n  a  nnn  n  n e   e a n   e e   a  a r   r         a  ae    r     a n   n        ea  ra   e   a  ee  n o      e    rn n   n       n   r   ea rrna   a     n   a
	---------------------------------------------------------------------------------------------------------------------------------------------------------------------
	DIVERSIDAD:  0.5
	   Generando con semilla: "iental, ingeniería electrónica, ingenier"